In [1]:
# setup
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm
from sklearn import datasets
from sklearn.datasets.base import Bunch
from IPython.display import Image
%matplotlib inline

# Background

Throwing out a set of theorems and definitions
## Bayes theorem:

\begin{align}
P(y|\vec{x}) = \frac{P(y)P(\vec{x}|y)}{P(\vec{x})}
\end{align}

## Mean Squared Error

\begin{equation}
\operatorname {MSE}={\frac  {1}{n}}\sum _{ {i=1} }^{n}({\hat  {Y_{i}}}-Y_{i})^{2}
\end{equation}

## Naive Bayes Classifier:
\begin{equation}
P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots x_n \mid y)}{P(x_1, \dots, x_n)} \\
\end{equation}
Using the naive independence assumption that
\begin{equation}
P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y)
\end{equation}
simplify
\begin{equation}
P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}{P(x_1, \dots, x_n)}
\end{equation}

${P(x_1, \dots, x_n)}$ is constant given the input.

\begin{align}
P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y) \\
\Downarrow \\
\hat{y} = \underset{y}{\operatorname{argmax}}\ P(y) \prod_{i=1}^{n} P(x_i \mid y) \\
\end{align}

Because float underflow, $log$ trick is often used
\begin{align}
\underset{y}{\operatorname{argmax}}\ P(y) \prod_{i=1}^{n} P(x_i \mid y)  & \propto \underset{y}{\operatorname{argmax}}\  log(P(y)) + \sum _{ {i=1} }^{n} log(P(x_{i}\vert y)) \\
 & = \underset{y}{\operatorname{argmax}}\ - (log(P(y)) + \sum _{ {i=1} }^{n} log(P(x_{i}\vert y))) \\
 & = \underset{y}{\operatorname{argmin}}\ - log(P(y)) - \sum _{ {i=1} }^{n} log(P(x_{i}\vert y)))
\end{align}

## $k$NN

In [6]:
euclid = lambda x,y: np.linalg.norm(x-y)

def majority_vote_classify(distance_function, data, target, x, k=3):
    distances = np.vectorize(distance_function)(data, x)
    idx = np.argpartition(distances, k)
    closest_classes = target[idx[:k]]
    cl = np.bincount(closest_classes).argmax()
    return cl

## $k$-Mean Classification Rule ($k$MC) for $k$NN:

General
\begin{align}
{\hat  {y}} &=\underset{y}{\operatorname{argmin}}\ \frac{\sum_{i=1}^k \text{dist}(\vec{x}, m_{iy})}{k}\\
\end{align}
k is independent of y, simplify
\begin{align}
{\hat  {y}} &=\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \text{dist}(\vec{x}, m_{iy})\\
\end{align}

Manhattan
\begin{align}
{\hat  {y}} &=\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sum_{j=1}^n{|(x_i - m_{ijy})|}
\end{align}

With euclids distance
\begin{align}
{\hat  {y}}  &=\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(x_i - m_{ijy})^2}}
\end{align}

#  Laat Classifier

I fantasize that $k$NN and NB are exactly the same. Can the classification rules be combined in an intuitive way?

## For funzies 

Consider Naive Bayes, $k$MC with manhattan and euclids distance:


\begin{align}
{\hat  {y}} &= \underset{y}{\operatorname{argmin}}\ P(y) \prod _{ {i=1} }^{n}P(x_{i}\vert y) \\
{\hat  {y}} &= \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sum_{j=1}^n{|(x_i - m_{ijy})|} \\
{\hat  {y}} &= \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(x_i - m_{ijy})^2}}
\end{align}

Observation:

$k$NN does not have something that looks like $P(y)$, could the "ball" arround $\vec{x}$ defined by $k$ be somewhat similar? 

If so, then 
\begin{equation} 
\prod _{ {i=1} }^{n}P(x_{i}\vert y)
\end{equation}
from naive bayes seems more interesting when combining the two concepts.


If its not, then maybe using $P(y)$ in $k$NN somehow could yield better results?

### Using $P(y)$ in $k$NN

\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ (1-P(y)) \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(x_i - m_{ijh})^2}}
\end{equation}

My gut feeling says that this probably has too much impact on estimation, and we would end up with classifying too many $\hat{y}$ with the highest $P(y)$ especially in cases where differences in $P(y)$ is high 

** TODO ** test on real data

### Subtracting likelihood from distance
Just subtracting the likelyhood from the distance, would that yield better classification?
Seems silly. 

\begin{equation}
{\hat  {y}} =\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sum_{j=1}^n{|(x_i - m_{ijy})|} - P(x_{i}\vert y)
\end{equation}

Wait, negative distances is probably bad. Lets fix that....

\begin{equation}
{\hat  {y}} =\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sum_{j=1}^n{|(x_i - m_{ijy})|} + (1-P(x_{i}\vert y))
\end{equation}

** TODO ** test on real data

### Using $p(y)$ and subtracting likelihood from distance

\begin{equation}
{\hat  {y}} =\underset{y}{\operatorname{argmin}}\ (1-P(y)) \sum_{i=1}^k \sum_{j=1}^n{|(x_i - m_{ijy})|}+ (1-P(x_{i}\vert y))
\end{equation}

** TODO ** test on real data

### Adding imaginary likelyhood to $m_{ijy}$?

Mapping $x$ to imaginary $x$-likelyhood:
\begin{equation}
\mathcal{I}(x|y) = x + P(x|y)i
\end{equation}

Then
\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(x_i - \mathcal{I}(m_{ijy}|y))^2}}
\end{equation}

Seems to me that this could "clean" the training data, unlikely features of $m_{iy}$ gives logner distances from the test vector $\vec{x}$. It could even be computed once when training.

** TODO ** test on real data

### Adding imaginary likelyhood to $x_i$?

\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(\mathcal{I}(x_i|y) - m_{ijy})^2}}
\end{equation}

This should penalize features in $\vec{x}$ that is unlikely to be a part of $y$

** TODO ** test on real data

### Adding imaginary likelyhood to $x_i$ and $m_{ijy}$?

\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(\mathcal{I}(x_i|y) - \mathcal{I}(m_{ijy}|y))^2}}
\end{equation}

Nah, this sort of negates the penalty for unlikely features in $y$. We can do better!

Lets take the complex conjugate $\overline{\mathcal{I}( \cdot, \cdot )}$ of one of them, it shouldn't matter which.
\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(\mathcal{I}(x_i|y) - \overline {\mathcal{I}(m_{ijy}|y)})^2}}
\end{equation}

This should penalize both unlikely features in $\vec{x}$ and pushing noisy training data away from $\vec{x}$

** TODO ** test on real data

### Using $P(y)$ *and* imaginary likelyhood

\begin{equation}
{\hat  {y}} = \underset{y}{\operatorname{argmin}}\ (1-P(y)) \sum_{i=1}^k \sqrt{\sum_{j=1}^n{(\mathcal{I}(x_i|y) - \overline {\mathcal{I}(m_{ijh}|y)})^2}}
\end{equation}

** TODO ** test on real data

### is the distance a metric?
\begin{equation}
d_{y}(\vec{x}, \vec{y}) = \sqrt{\sum_{j=1}^n{(\mathcal{I}_{y}(x_i) - \overline {\mathcal{I}_{y}(y_i)})^2}}
\end{equation}


1. $d_{y}(x,y)\geq 0$ non-negativity 👍
2. $d_{y}(x,x) = 0$ identity 👎
3. $d_{y}(x,y) = d_{y}(y,x) $ symmetry 👍
4. $d_{y}(x,z) \leq d_{y}(x,y) + d_{y}(y,z)$  triangle inequality ? 👎

### MSE as distance?

\begin{equation}
\operatorname {MSE}={\frac  {1}{n}}\sum _{ {i=1} }^{n}({\hat  {Y_{i}}}-Y_{i})^{2}
\end{equation}

\begin{align}
{\hat  {y}} &= \underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k {\frac  {1}{n}}\sum _{ {i=1} }^{n}({x_{i}}-m_{ijy})^{2} \\
&=\underset{y}{\operatorname{argmin}}\ \sum_{i=1}^k \sum _{ {i=1} }^{n}({x_{i}}-m_{ijy})^{2}
\end{align}

## More Funzies

### Requirement: 

All features must be normalized so that all $x$ in $X$ is between $[0,1]$

### consider:

Generalized Mean:
\begin{align}
\mathrm {m}_p(\vec{x})=\left({\frac  {1}{n}}\sum _{ {i=1} }^{n}x_{i}^{p}\right)^{ { {\frac  {1}{p}}}}
\end{align}

Generalized Mean Distance:
\begin{align}
\mathrm {md}_p(\vec{x}, \vec{y})=\left({\frac  {1}{n}}\sum _{ {i=1} }^{n}(x_{i}-y_{i})^{p}\right)^{ { {\frac  {1}{p}}}}
\end{align}

Root Mean Squared Distance (RMSD):
\begin{align}
\mathrm {rmsd}(\vec{x}, \vec{y}) &= md_2(\vec{x}, \vec{y})\\ 
\mathrm {rmsd}(\vec{x}, \vec{y}) &={\sqrt { {\frac {1}{n}}\left(\sum _{ {i=1} }^{n}(x_{i}-y_{i})^{2}\right)}}
\end{align}

$k$ Mean RMSD
\begin{align}
\mathrm {krmsd}(\vec{x}, \vec{y})=\frac{\sum_{i=1}^k \mathrm{rmsd}(\vec{x}, \vec{y})}{k}
\end{align}
### try:

\begin{align}
{\hat  {y}} & = \underset{y}{\operatorname{argmin}}\ P(y) \prod _{ {i=1} }^{n}P(x_{i}\vert y)
\end{align}

I know, I'm breaking all the rules! 

\begin{align}
{\hat  {y}} & = \underset{y}{\operatorname{argmin}}\ \frac{P(y)\prod _{ {i=1} }^{n}P(x_{i}\vert y)}{1 - \mathrm{krmsd}(\vec{x}, \vec{m_{iy}})} 
\end{align}